**Microsoft Malware Prediction**



The goal of this competition is to predict a Windows machine’s probability of getting infected by various families of malware, based on different properties of that machine. The telemetry data containing these properties and the machine infections was generated by combining heartbeat and threat reports collected by Microsoft's endpoint protection solution, Windows Defender.

Each row in this dataset corresponds to a machine, uniquely identified by a MachineIdentifier. HasDetections is the ground truth and indicates that Malware was detected on the machine. Using the information and labels in train.csv, you must predict the value for HasDetections for each machine in test.csv.

The sampling methodology used to create this dataset was designed to meet certain business constraints, both in regards to user privacy as well as the time period during which the machine was running. Malware detection is inherently a time-series problem, but it is made complicated by the introduction of new machines, machines that come online and offline, machines that receive patches, machines that receive new operating systems, etc. While the dataset provided here has been roughly split by time, the complications and sampling requirements mentioned above may mean you may see imperfect agreement between your cross validation, public, and private scores! Additionally, this dataset is not representative of Microsoft customers’ machines in the wild; it has been sampled to include a much larger proportion of malware machines.

***Can you predict if a machine will soon be hit with malware?***

In [ ]:
import pandas as pd
import numpy as np
import gc

# Gradient Boosting
import lightgbm as lgb

# Scikit-learn
from sklearn.metrics import average_precision_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# Graphics
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Metrics
from sklearn.metrics import average_precision_score, roc_auc_score, mean_absolute_error

import os
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

**1) Loading the data**

The data provided is huge hence we need to take special care here to load this data .
A solution is to specify types, to gain memory (for example switching from float64 to float32)

*Thanks to **Theo Viel** for the kernel to help loading the data by reducing the memory usage.
https://www.kaggle.com/theoviel/load-the-totality-of-the-data*

In [ ]:
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
from copy import deepcopy
dtypes1 = deepcopy(dtypes) 
del dtypes1['HasDetections']
dtypes1

I will be reading only a part of the training data (2000000 rows i.e 1/4th of the total training data)

In [ ]:
malware_data=pd.read_csv('../input/train.csv',nrows=2000000, dtype=dtypes)
malware_data = reduce_mem_usage(malware_data)

In [ ]:
malware_data_test=pd.read_csv('../input/test.csv', dtype=dtypes1)
malware_data_test = reduce_mem_usage(malware_data_test)

**2) EDA**

In [ ]:
malware_data.head(10) #Checking the  training data

Looking for the percentage of null values present in the columns.

In [ ]:
(malware_data.isnull().sum()/malware_data.shape[0]).sort_values(ascending=False)

Since 'PuaMode' column has almost all data in it null i will drop it and ,along with it i will also drop the id column.

In [ ]:
remove_cols = ['PuaMode','MachineIdentifier']
malware_data.drop(remove_cols, axis=1, inplace=True)

malware_data_test.drop(remove_cols, axis=1, inplace=True)

In [ ]:
malware_data.head(10) #Checking the  training data

In [ ]:
malware_data_test.head(10) #Checking the test data

The data has a mixture of different data types but after observation i found that only few columns have meaningful numeric type data rest are binary and categorical data types.

In [ ]:
true_numerical_columns = [
    'Census_ProcessorCoreCount',
    'Census_PrimaryDiskTotalCapacity',
    'Census_SystemVolumeTotalCapacity',
    'Census_TotalPhysicalRAM',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches',
    'Census_InternalPrimaryDisplayResolutionHorizontal',
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_InternalBatteryNumberOfCharges'
]

In [ ]:
binary_variables = [c for c in malware_data.columns if malware_data[c].nunique() == 2]
binary_variables

In [ ]:
categorical_columns = [c for c in malware_data.columns 
                       if (c not in true_numerical_columns) & (c not in binary_variables)]

In [ ]:
categorical_columns

**3) Feature Engineering**


In [ ]:
train_df=malware_data
X_test=malware_data_test
del malware_data
del malware_data_test

In [ ]:
gc.collect() #Garbage Collector to release unreferenced memory

Applying label encoding on all the categorical data columns.

In [ ]:
from sklearn.preprocessing import LabelEncoder
train_length = train_df.shape[0]
for col in (categorical_columns):
    combined_data = pd.concat([train_df[col], X_test[col]])
    combined_data, _ = pd.factorize(combined_data)
    combined_data = pd.Series(combined_data).astype('int32')
    train_df[col] = combined_data.iloc[:train_length].values
    X_test[col] = combined_data.iloc[train_length:].values
    train_df[col] = train_df[col].fillna(0)
    X_test[col] = X_test[col].fillna(0)
    del combined_data

In [ ]:
train_df = reduce_mem_usage(train_df)
X_test = reduce_mem_usage(X_test)

**4) LightGBM Model **

In [ ]:
param = {   
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'nthread': 4,
        'learning_rate': 0.05,
        'max_depth': 5,
        'num_leaves': 40,
        'sub_feature': 0.9,
        'sub_row':0.9,
        'bagging_freq': 1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'random_state': 15
        }

I am using StratifiedKFold technique for Cross-Validation with 5 folds.

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=15) 

In [ ]:
gc.collect()

In [ ]:
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(len(X_test))
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['HasDetections']]
    
for n_fold, (train_idx, valid_idx) in enumerate(kf.split(train_df[feats], train_df['HasDetections'])):
    print(n_fold)
    trn_data = lgb.Dataset(train_df.iloc[train_idx][feats], label=train_df['HasDetections'].iloc[train_idx])    
    val_data = lgb.Dataset(train_df.iloc[valid_idx][feats], label=train_df['HasDetections'].iloc[valid_idx])
        
    clf = lgb.train(param, trn_data,5120, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)   

    oof_preds[valid_idx] = clf.predict(train_df.iloc[valid_idx][feats], num_iteration=clf.best_iteration) 
    
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = feats
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = n_fold + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    
    # we perform predictions by chunks
    initial_idx = 0
    chunk_size = 100000
    current_pred = np.zeros(len(X_test))
    
    while initial_idx < X_test.shape[0]:
        final_idx = min(initial_idx + chunk_size, X_test.shape[0])
        idx = range(initial_idx, final_idx)
        current_pred[idx] = clf.predict(X_test.iloc[idx][feats], num_iteration=clf.best_iteration)
        initial_idx = final_idx
        
    sub_preds += current_pred / 5

print('Full AUC score %.6f' % roc_auc_score(train_df['HasDetections'], oof_preds))

**5) Feature Importance**

Below visualized is the importance (for the above model ) of each feature in the dataset.

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:150].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,28))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
plt.savefig('FI.png')

**6) Submission**

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission['HasDetections'] = sub_preds
sample_submission.to_csv('submission.csv', index=False)